In [13]:
# Basic Tools
import pandas as pd
import numpy as np



from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, cross_validate



In [14]:
data = pd.read_csv('../data/processed/data_processed.csv')
data_prep = data.copy()

In [15]:
data_prep.shape

(6362620, 11)

In [16]:
data_prep.isna().sum()

step                0
type                0
amount              0
name_orig           0
oldbalance_org      0
newbalance_orig     0
name_dest           0
oldbalance_dest     0
newbalance_dest     0
is_fraud            0
is_flagged_fraud    0
dtype: int64

In [17]:
data_prep.head()

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud
0,1,0,9839.64,1231006815,170136.0,160296.36,1979787155,0.0,0.0,0,0
1,1,0,1864.28,1666544295,21249.0,19384.72,2044282225,0.0,0.0,0,0
2,1,1,181.00,1305486145,181.0,0.00,553264065,0.0,0.0,1,0
3,1,2,181.00,840083671,181.0,0.00,38997010,21182.0,0.0,1,0
4,1,0,11668.14,2048537720,41554.0,29885.86,1230701703,0.0,0.0,0,0


In [18]:
data_prep.columns

Index(['step', 'type', 'amount', 'name_orig', 'oldbalance_org',
       'newbalance_orig', 'name_dest', 'oldbalance_dest', 'newbalance_dest',
       'is_fraud', 'is_flagged_fraud'],
      dtype='object')

In [20]:
data_prep.dtypes

step                  int64
type                  int64
amount              float64
name_orig             int64
oldbalance_org      float64
newbalance_orig     float64
name_dest             int64
oldbalance_dest     float64
newbalance_dest     float64
is_fraud              int64
is_flagged_fraud      int64
dtype: object

# Encoding

In [21]:
# map_type = {'PAYMENT': 0,'TRANSFER':1,'CASH_OUT': 2,'DEBIT': 3,'CASH_IN': 4}
# data_prep['type'] = data_prep['type'].map(map_type)

# Feature Selection

In [24]:
SELECTED_FEATURES = ['step', 'type', 'amount', 'name_orig', 'oldbalance_org',
       'newbalance_orig', 'name_dest', 'oldbalance_dest', 'newbalance_dest', 'is_flagged_fraud']




TARGET = 'is_fraud'

In [25]:
data_prep[SELECTED_FEATURES]

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_flagged_fraud
0,1,0,9839.64,1231006815,170136.00,160296.36,1979787155,0.00,0.00,0
1,1,0,1864.28,1666544295,21249.00,19384.72,2044282225,0.00,0.00,0
2,1,1,181.00,1305486145,181.00,0.00,553264065,0.00,0.00,0
3,1,2,181.00,840083671,181.00,0.00,38997010,21182.00,0.00,0
4,1,0,11668.14,2048537720,41554.00,29885.86,1230701703,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...
6362615,743,2,339682.13,786484425,339682.13,0.00,776919290,0.00,339682.13,0
6362616,743,1,6311409.28,1529008245,6311409.28,0.00,1881841831,0.00,0.00,0
6362617,743,2,6311409.28,1162922333,6311409.28,0.00,1365125890,68488.84,6379898.11,0
6362618,743,1,850002.52,1685995037,850002.52,0.00,2080388513,0.00,0.00,0


# Normalization

In [26]:
log_columns = data_prep[SELECTED_FEATURES].skew().sort_values(ascending=False)
log_columns = log_columns.loc[log_columns > 0.75]
print('Columns/Skew\n',log_columns)   

Columns/Skew
 is_flagged_fraud    630.603629
amount               30.993949
oldbalance_dest      19.921758
newbalance_dest      19.352302
oldbalance_org        5.249136
newbalance_orig       5.176884
dtype: float64


In [27]:
y = data_prep[TARGET]
X = data_prep[SELECTED_FEATURES]

In [28]:
seed=7
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state=seed,stratify=y)#, stratify=y
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((5090096, 10), (5090096,), (1272524, 10), (1272524,))

In [31]:
data_prep['is_fraud'].value_counts()

is_fraud
0    6354407
1       8213
Name: count, dtype: int64

In [32]:
y_train['is_fraud'].value_counts()

KeyError: 'is_fraud'

In [34]:
# Use numpy.unique para obter valores únicos
unique_values = np.unique(y_train)

# Use numpy.bincount para contar as ocorrências de cada valor
value_counts = np.bincount(y_train)

# Crie um dicionário para mapear os valores únicos para suas contagens
value_counts_dict = {unique_value: count for unique_value, count in zip(unique_values, value_counts)}

print(value_counts_dict)

{0: 5083526, 1: 6570}


In [35]:
# Use numpy.unique para obter valores únicos
unique_values = np.unique(y_val)

# Use numpy.bincount para contar as ocorrências de cada valor
value_counts = np.bincount(y_val)

# Crie um dicionário para mapear os valores únicos para suas contagens
value_counts_dict = {unique_value: count for unique_value, count in zip(unique_values, value_counts)}

print(value_counts_dict)

{0: 1270881, 1: 1643}


In [36]:
number_folds = 4
Kfold = KFold(n_splits=number_folds, shuffle=True, random_state=seed)